In [87]:
import numpy as np
import pickle

In [88]:
def count_num_pred(file):
    with open(file, "r") as f:
        len_sents = []
        n_pred = 0
        line = f.readline()
        while True:
            if line == "":
                break
            word = line.split("\t")
            if word[0] != "\n":
                if word[13]!= "_" and word[13]!= "_\n":
                    n_pred += 1
                line = f.readline()
            else:  
                len_sents.append(n_pred)
                n_pred = 0
                line = f.readline()

        return len_sents                    


arg_pos = pickle.load(open("./predictions_srl_det/arguments.pickle", "rb"))

inv_arg = {}
for k in arg_pos:
    inv_arg[arg_pos[k]] = k

In [89]:
### load testverbs
srl_arg = []
detec_arg = []
for i in range(30):
    srl_arg.append(np.load("./predictions_srl_det/srl_"+str(i)+"_test.npy"))
    detec_arg.append(np.load("./predictions_srl_det/detection_"+str(i)+"_test.npy"))
    
srl_arg = np.concatenate(srl_arg, axis=0)
detec_arg = np.concatenate(detec_arg, axis=0)

file_from = "./data/TestData/test.csv"
num_pred = count_num_pred(file_from)


In [90]:
 def make_file_with_srl(file, to_file, srl, detection, inv_arg, num_pred):
        with open(file, "r") as f , open(to_file, "w") as to_f:
            sentence, sent_ind = 0, 0
            w_ind = 0
            a = np.exp(detection)
            probs = a / np.expand_dims(np.sum(a, axis=2),axis=-1)
            is_arg = (probs[:,:,1] > 0.5)
            line = f.readline()
            while True:
                if line == "":
                    break
                word = line.split("\t")
                if word[0] != "\n":
                    word[-1] = word[-1].split()[0]
                    if num_pred[sentence]!=0:
                        for pred in range(num_pred[sentence]):
                            if is_arg[sent_ind , w_ind] == 1:
                                word.append(inv_arg[srl[sent_ind, w_ind]])
                                sent_ind += 1
                            else:
                                word.append("_")
                                sent_ind += 1
                        
                        
                    to_f.write("\t".join(word)+"\n")
                    line = f.readline()
                    w_ind += 1
                    sent_ind -= num_pred[sentence]
                    

                else:                    
                    w_ind = 0
                    sent_ind += num_pred[sentence]
                    sentence += 1
                    to_f.write("\n")
                    line = f.readline()
                    
                    


In [91]:
make_file_with_srl(file_from, "./test_with_arguments.txt", srl_arg, detec_arg, inv_arg, num_pred)

saving testverbs data

In [92]:
### load testverbs
srl_arg = []
detec_arg = []
for i in range(6):
    srl_arg.append(np.load("./predictions_srl_det/srl_testverbs"+str(i)+".npy"))
    detec_arg.append(np.load("./predictions_srl_det/detection_testverb"+str(i)+".npy"))
    
srl_arg = np.concatenate(srl_arg, axis=0)
detec_arg = np.concatenate(detec_arg, axis=0)

file_from = "./testverbs_with_predicate.txt"
num_pred = count_num_pred(file_from)


In [93]:
make_file_with_srl(file_from, "./testverbs_with_arguments.txt", srl_arg, detec_arg, inv_arg, num_pred)